In [ ]:
!git clone https://github.com/JBiiiS/JBiS.git

import os
os.chdir('JBiS') # Adjust the path to your project root

# Fetch the most recent commits from your GitHub repository
!git pull origin main

pip install torchsde

pip install torchcde

pip install -U  finance-datareader


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import FinanceDataReader as fdr
import random
import seaborn as sns
from tqdm import tqdm
from torch.utils.data import TensorDataset, DataLoader
import importlib as imp

from base.base_config import BaseConfig
from base.data_process import *

from neural_sde_gan.neural_sde_config import NeuralSDEConfig
from neural_sde_gan.sde_generator import SDEGenerator
from neural_sde_gan.cde_discriminator import CDEDiscriminator
from neural_sde_gan.neural_sde_gan_model import NeuralSDEGAN
from neural_sde_gan.sde_trainer import _gradient_penalty, _step_D, _step_G

In [ ]:
config =  NeuralSDEConfig(project_name = 'Neural_SDE', experiment_name= '260224_1641', num_epochs=100)

if hasattr(config, 'device'):
    device = config.device
else:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Device set to: {device}")

In [ ]:
price_df = fdr_data_wo_ticker(TARGET_N_STOCKS=config.num_assets, START_DATE='2010-01-01', END_DATE='2025-12-31')

price_df = clean_price_data(price_df, no_minus=True)

ln_price, ln_price_diff = log_data(price_df)

ln_price = clean_price_data(ln_price, no_minus=True)
ln_price_diff = clean_price_data(ln_price_diff, no_minus=False)

In [ ]:
processor = To_TensorSet(config)
tensor_set = processor.process(ln_price_diff)


train_tensor = tensor_set['train']['BTD']  # Shape: [N_train, 30, 500]
test_tensor  = tensor_set['test']['BTD']   # Shape: [1, 30, 500]

train_dataset = TensorDataset(train_tensor)
test_dataset  = TensorDataset(test_tensor)

# [4] DataLoader 생성
train_loader = DataLoader(
    train_dataset,
    batch_size=config.batch_size,
    shuffle=True,            
    drop_last=True             
)


test_loader = DataLoader(
    test_dataset,
    batch_size=1,             
    shuffle=False
)

# -------------------------------------------------------
# Testify
# -------------------------------------------------------
print(f"Train Data Shape: {train_tensor.shape}")
print(f"Train Loader Batches: {len(train_loader)}")


for batch in train_loader:
    x_batch = batch[0]
    print(f"Input Batch Shape: {x_batch.shape}") 
    break


In [ ]:
model = NeuralSDEGAN(
    config=config, 
    generator=SDEGenerator, 
    discriminator=CDEDiscriminator
).to(config.device)


optimizer_D = optim.Adam(
    model.discriminator.parameters(),
    lr=config.learning_rate,
    weight_decay=config.weight_decay,
    betas=(0.5, 0.9)
)

optimizer_G = optim.Adam(
    model.generator.parameters(),
    lr=config.learning_rate,
    weight_decay=config.weight_decay,
    betas=(0.5, 0.9)
)

print("Model and Optimizers initialized successfully.")

In [ ]:
print("Starting SDE-GAN Training...")

# 손실 기록용 딕셔너리
history = {'D_loss': [], 'G_loss': []}

for epoch in range(config.num_epochs):
    epoch_d_loss = 0.0
    epoch_g_loss = 0.0
    batches = 0
    
    # tqdm으로 진행률 표시
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{config.num_epochs}", leave=False)
    
    for batch in progress_bar:
        real_x = batch[0].to(config.device) 
        
        # 1. Discriminator 
        for _ in range(config.epoch_for_D):
            d_loss = _step_D(model, real_x, optimizer_D, config)
            
        epoch_d_loss += d_loss
        
        # 2. Generator 
        for _ in range(config.epoch_for_G):
            g_loss = _step_G(model, optimizer_G)
            
        epoch_g_loss += g_loss
        batches += 1
        
        # Loss
        progress_bar.set_postfix({
            'D_loss': f"{d_loss:.4f}", 
            'G_loss': f"{g_loss:.4f}"
        })
        
    # 에폭 평균 손실 기록
    avg_d_loss = epoch_d_loss / batches
    avg_g_loss = epoch_g_loss / batches
    history['D_loss'].append(avg_d_loss)
    history['G_loss'].append(avg_g_loss)
    
    # 10 에폭마다 로그 출력 및 체크포인트 저장
    if (epoch + 1) % 2 == 0:
        print(f"Epoch [{epoch+1}/{config.num_epochs}] | D Loss: {avg_d_loss:.4f} | G Loss: {avg_g_loss:.4f}")
        
    
print("Training Completed!")

In [ ]:
plt.style.use('seaborn-v0_8-whitegrid')

# ---------------------------------------------------------
# 3. Model Evaluation
# ---------------------------------------------------------
model.eval()

# 테스트 데이터셋에서 하나의 배치 추출
test_batch = next(iter(test_loader))
real_test_path = test_batch[0].to(config.device) # (1, steps, output_dim)

# Generator를 통해 가상의 궤적 생성
with torch.no_grad():
    x_hat_full, x_hat_matched = model.generate()

# 시각화를 위해 CPU 및 Numpy 배열로 변환
times = config.sde_times_matched.cpu().numpy()
real_sample = real_test_path[0].cpu().numpy()    # 첫 번째 샘플
fake_sample = x_hat_matched[0].cpu().numpy()     # 첫 번째 샘플

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# [Plot 1] WGAN-GP Training Loss
axes[0].plot(history['D_loss'], label='Critic (D) Loss', color='crimson', alpha=0.8)
axes[0].plot(history['G_loss'], label='Generator (G) Loss', color='navy', alpha=0.8)
axes[0].set_title('WGAN-GP Training Loss Convergence')
axes[0].set_xlabel('Epochs')
axes[0].set_ylabel('Loss')
axes[0].legend()

# [Plot 2] Real vs Fake Trajectory (첫 번째 자산 기준)
asset_idx = 0 
axes[1].plot(times, real_sample[:, asset_idx], label='Real Market Path', color='black', linewidth=1.5)
axes[1].plot(times, fake_sample[:, asset_idx], label='Generated SDE Path', color='darkorange', linestyle='--', linewidth=1.5)
axes[1].set_title(f'Continuous Path Evaluation (Asset {asset_idx})')
axes[1].set_xlabel('Time (t)')
axes[1].set_ylabel('Log Return')
axes[1].legend()

plt.tight_layout()
plt.show()